### Simulation results

Once your simulation has finished, you probably want to analyse the results with for example
the [ThreediToolbox](https://docs.3di.live/d_qgis_plugin.html) or
[threedigrid](https://threedigrid.readthedocs.io/en/latest/readme.html#). In this notebook
we'll show you how you can download the result files of a 3Di simulation.

Let's first create the client to connect to the threedi-api:

In [ ]:
import requests
from getpass import getpass
from pathlib import Path
from openapi_client import ThreedimodelsApi, SimulationsApi, FilesApi
from threedi_api_client.threedi_api_client import ThreediApiClient

In [ ]:
# Provide authentication details
API_HOST = "https://api.3di.live/v3.0"
USERNAME = input("Username")
PASSWORD = getpass("Password")

config = {
    "API_HOST": API_HOST,
    "API_USERNAME": USERNAME,
    "API_PASSWORD": PASSWORD
}

api_client = ThreediApiClient(config=config)

Now let's select a simulation we're interested in. Make sure it is finished simulating.
We will select the simulation we created in the [previous](./simulation_with_events.ipynb)
notebook.


In [ ]:
simulation_api = SimulationsApi(api_client)

simulation = simulation_api.simulations_list(
    name="my simulation with events", user__username=USERNAME
).results[-1]
status = simulation_api.simulations_status_list(simulation.id)

print(simulation)
print(f"status: {status}")
assert status.name == 'finished'


Now that we have our finished simulation lets see which result files are available:

In [ ]:
result_files = simulation_api.simulations_results_files_list(simulation.id)

for result in result_files.results:
    print(result)

Let's download all the results so we can analyse them locally:

In [ ]:
download_folder = Path(f'Results {simulation.name}')
download_folder.mkdir(exist_ok=True)

for file in result_files.results:
    download_url = simulation_api.simulations_results_files_download(
        id=file.id, simulation_pk=simulation.id
    )

    file_path = download_folder / file.filename
    r = requests.get(download_url.get_url)
    with open(file_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

    print(f"Finished downloading {file.filename}")


The ThreediToolbox and threedigrid also require the "gridadmin.h5" file of the simulation.
This is a model specific file so it's under the threedimodels-api. We'll also download this file:

In [ ]:
model_api = ThreedimodelsApi(api_client)

threedi_model_id = simulation.threedimodel_id
download_url = model_api.threedimodels_gridadmin_download(threedi_model_id)

file_path = download_folder / "gridadmin.h5"
r = requests.get(download_url.get_url)
with open(file_path, "wb") as f:
    for chunk in r.iter_content(chunk_size=8192):
        f.write(chunk)

print(f"Finished downloading gridadmin.h5")